In [8]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 26 14:23:30 2022
@author: Guillaume PECH
"""

# Import necessary libraries
import os, mne, csv, re
import numpy as np
from mne.channels import make_standard_montage
from glob import glob
from mne_bids import BIDSPath, write_raw_bids
from pyprep.find_noisy_channels import NoisyChannels

# Set the source directory path for the raw data
root_source = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/raw'

# Set the folder for recreate the data in bids format downsampled 
root_bids = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/bids/'

# Set the derivatives directory path
root_derivatives = root_source.split('/raw')[0] + '/derivatives'

# Set the image directory path
root_image = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/images_prep_eeg/'

# Change the current working directory to the derivatives directory
os.chdir(root_derivatives)

# Get a list of all file paths with .bdf extension in the source directory
all_files_path = sorted(glob(f'{root_source}/*V2.bdf'), key=len)


In [49]:
%%capture

# Define a dictionary with participant numbers as keys and a list of bad channels as values + a list of bad ica components

prtp_chs_out = {}


prtp_ica_out = {}

# Loop over each file path in the list of all_files_path
for part in all_files_path:
        # Extract the participant number from the file path
        n_part = re.split('raw|_SIV2.bdf', part)[1][1:]  # e.g., '001'
        # Read the raw data from the BDF file
        raw = mne.io.read_raw_bdf(part)
        
        bidspath = BIDSPath(subject=n_part, task='socInfluenceV2', datatype='eeg', root=root_bids)
        
        # Load the data into memory
        raw.load_data()

        events= mne.find_events(raw, shortest_event=1)
        try :
            raw.crop(((events[1,0]/2048))-3, (events[-1,0]/2048)+3)
        except : 
            pass

        events= mne.find_events(raw, shortest_event=1)
        # # Resample the data by downsampling to 1/4th of the original sample rate
        raw, events = raw.resample(sfreq=raw.info['sfreq'] / 4, events= events)
        
        event_id = { 'vizualisation/forced/authority/yes':1,'vizualisation/forced/authority/no':2,'vizualisation/forced/group/yes':3,'vizualisation/forced/group/no':4,
        'vizualisation/forced/indiv/yes':5,'vizualisation/forced/indiv/no':6,'vizualisation/free/authority-yes/group-yes':7,'vizualisation/free/authority-yes/group-no':8,'vizualisation/free/authority-no/group-yes':11,
        'vizualisation/free/authority-yes/indiv-yes':9,'vizualisation/free/authority-yes/indiv-no':10, 'vizualisation/free/authority-no/group-no':12,'vizualisation/free/authority-no/indiv-yes':13, 'vizualisation/free/authority-no/indiv-no':14,
        'vizualisation/free/group-yes/indiv-yes':15, 'vizualisation/free/group-yes/indiv-no':16,'vizualisation/free/group-no/indiv-yes':17, 'vizualisation/free/group-no/indiv-no':18,
        'selection/forced/authority/yes':19,'selection/forced/authority/no':20,'selection/forced/group/yes':21,'selection/forced/group/no':22,'selection/forced/indiv/yes':23,'selection/forced/indiv/no':24,
        'selection/free/authority/yes':25,'selection/free/authority/no':26,'selection/free/group/yes':27,'selection/free/group/no':28,'selection/free/indiv/yes':29,'selection/free/indiv/no':30}# Define event dictionary
    
        events = events[np.isin(events[:, 2], list(event_id.values()))]
        raw.add_events(events,replace=True)


        try:
            raw.set_channel_types({'EXG1': 'misc', 'EXG2': 'misc', 'EXG3': 'eog', 'EXG4': 'eog'})
        except:
            pass
        # Set the 32 system BioSemi channel positions on the data
        montage = make_standard_montage('biosemi32')
        new_names = dict(zip(raw.ch_names, montage.ch_names))
        raw.rename_channels(new_names)
        raw.set_montage(montage)

        write_raw_bids(raw, bidspath, events=events, event_id=event_id, format='BrainVision', allow_preload=True,overwrite=True)

        # Apply a bandpass filter to the raw data 
        raw.filter(l_freq=1, h_freq=30, picks='all', n_jobs=-1)

        # Interpolate bad channels for participants in the prtp_chs_out list
        if n_part in list(prtp_chs_out):
            raw.info['bads'] = prtp_chs_out[n_part]
            raw = raw.interpolate_bads()
        else :
            noisy= NoisyChannels(raw)
            noisy.find_all_bads()
            bad_chs = noisy.get_bads()
            raw.info['bads'] = bad_chs
            raw = raw.interpolate_bads()

            prtp_chs_out[n_part]=bad_chs

        # Perform Independent Component Analysis (ICA) on the copied data
        ica = mne.preprocessing.ICA(random_state=21, max_iter='auto')
        ica.fit(raw)

        # Set the channel types for EOG channels based on the participant
        raw.set_channel_types({'Fp1': 'eog', 'Fp2': 'eog'})

        # Selecting bad components ICA for participants in the prtp_ica_out list or calculating bad components

        if n_part in list(prtp_ica_out):
            eog_indices = prtp_ica_out[n_part]
        else : 
            # Find ICs (Independent Components) that match the EOG pattern
            eog_indices, eog_scores = ica.find_bads_eog(raw, measure='correlation', threshold=0.5)
            prtp_ica_out[n_part]=eog_indices

        ica.exclude = eog_indices

        # Set the channel types back to EEG channels 
        raw.set_channel_types({'Fp1': 'eeg', 'Fp2': 'eeg'})

        # If there are EOG components to exclude, plot diagnostics and save the figures
        if len(eog_indices) > 0:
            ica.apply(raw)
            fig = ica.plot_components(picks=np.arange(ica.n_components_), show = False) #create figures of the components with the one removed
            fig.savefig(f"{root_image}sub-{n_part}_SIV2_ICA.png")

        del ica

        # Save the preprocessed raw data
        raw.save(f"sub-{n_part}_SIV2_raw.fif", overwrite=True)

        del raw

# create the txt file

with open('list_bad_channels.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_chs_out) + '\n')
    
with open('list_bad_ica.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_ica_out) + '\n')    


In [12]:
chs_out = []
for i in prtp_ica_out:
    chs_out.append(len(prtp_ica_out[i]))

In [13]:
import numpy as np 
print(np.mean(chs_out),np.std(chs_out))

2.1184210526315788 0.32310603046864933


In [2]:
prtp_chs_out = {'154': ['T7'], '156': ['PO3', 'T7', 'PO4'], '157': ['PO3', 'T7', 'Pz'], '159': ['Pz', 'O1', 'PO3', 'T7', 'P7', 'CP1'], '166': ['PO3'], '169': [], '170': ['O1', 'PO3'], '171': ['CP6', 'PO3'], '172': ['Pz', 'PO3', 'CP5', 'T7', 'CP1'], '173': [], '175': ['Pz', 'P8', 'O1', 'O2', 'PO3', 'T7'], '176': ['PO3'], '177': ['PO3'], '178': ['P3', 'PO3', 'T7', 'Pz'], '179': ['P7', 'O2', 'PO3'], '180': ['CP6', 'PO3'], '181': ['P7', 'P3', 'PO3', 'CP2'], '183': ['PO3'], '184': ['T8', 'F7', 'F4', 'PO3', 'T7', 'F3'], '728': ['PO3'], '729': ['PO3'], '730': [], '731': ['Pz'], '732': ['Pz'], '733': ['PO3'], '734': ['PO3', 'Pz'], '735': [], '736': [], '737': [], '739': ['P7', 'T8', 'T7'], '740': ['P3'], '741': ['CP6', 'PO3', 'T8', 'PO4'], '742': ['P7', 'PO3', 'PO4'], '743': ['T8', 'PO3', 'Pz'], '744': ['Pz'], '746': ['PO4', 'Pz'], '747': ['O1'], '748': ['PO3'], '931': [], '932': ['P3', 'PO3'], '933': ['P3', 'PO3'], '934': ['Pz'], '935': ['PO3', 'Pz'], '937': [], '938': ['PO3'], '940': ['PO3', 'Pz'], '941': ['P7', 'O1'], '942': ['PO3'], '944': ['Pz'], '945': ['P3', 'PO3', 'T7'], '946': ['PO3', 'Pz'], '947': ['T8', 'Fp2'], '948': ['PO3', 'Pz'], '949': ['P3', 'F7', 'Pz'], '950': ['O2', 'P4', 'Oz', 'CP5'], '951': ['T8', 'Fp1', 'P4', 'PO3', 'T7', 'PO4'], '952': ['O2', 'PO3', 'Oz', 'Pz'], '953': ['T7'], '954': ['PO3'], '955': ['P4', 'PO3'], '956': ['P3', 'CP2', 'CP1'], '957': ['O1', 'P3', 'PO3'], '958': [], '155': ['Fp1', 'Fp2', 'PO3'], '160': ['PO3'], '161': [], '162': ['PO3', 'T7'], '163': [], '164': ['Fp1', 'Fp2', 'Pz', 'PO4'], '165': ['T7'], '167': ['Oz'], '174': ['PO3', 'PO4'], '182': ['P3'], '745': ['PO3'], '936': ['Fp1', 'Fp2', 'F7'], '939': []}

prtp_ica_out ={'154': [0, 2], '156': [0, 2], '157': [0, 1], '159': [0, 1], '162': [0, 2], '166': [1, 2], '169': [0, 1], '170': [3, 1], '171': [0, 1, 2], '172': [0, 3], '173': [1, 3], '175': [0, 1], '176': [0, 2], '177': [0, 1], '178': [0, 4], '179': [0, 2], '180': [1, 4], '181': [0, 2], '183': [0, 2, 3], '184': [1, 4], '728': [0, 2], '729': [0, 2], '730': [0, 2], '731': [2, 4], '732': [1, 0], '733': [0, 1], '734': [0, 5], '735': [14, 5, 2], '736': [1, 2], '737': [0, 1], '739': [1, 0], '740': [0, 2], '741': [0, 3, 1], '742': [0, 1], '743': [5, 17], '744': [0, 3], '746': [0, 5], '747': [8, 9], '748': [0, 3], '931': [0, 1], '932': [1, 4], '933': [0, 1], '934': [0, 2], '935': [1,3], '937': [0, 1], '938': [0, 1], '940': [0, 2], '941': [0, 1], '942': [0, 2], '944': [0, 1], '945': [0, 1], '946': [2, 3], '947': [2, 0, 1], '948': [0, 1], '949': [0, 1], '950': [0, 3, 4], '951': [2, 4], '952': [1,12], '953': [1, 2], '954': [0, 2, 1], '955': [0, 4, 3], '956': [1, 2], '957': [1, 5, 7], '958': [0, 1],'155': [4,6], '160': [1,3], '161': [2,4], '163': [2,3], '164': [6,10], '165': [3, 6], '167': [0, 3], '174': [4, 5], '182': [1, 26], '745': [2, 22], '936': [1,6], '939': [22, 17]}

In [5]:

# Import necessary libraries
import os, mne, csv, re
import numpy as np
from mne.channels import make_standard_montage
from glob import glob
from mne_bids import BIDSPath, write_raw_bids, read_raw_bids

from pyprep.find_noisy_channels import NoisyChannels
# Set the source directory path for the sampled bids data / some participant segment have been marked as bad due to poor ICA components visualization
root_source = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/bids/'

# Set the derivatives directory path
root_derivatives = root_source.split('/bids')[0] + '/derivatives_with_tfr'

# Set the image directory path
root_image = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/images_prep_eeg/'

root_bids = 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/1-bids/'


# Change the current working directory to the derivatives directory
os.chdir(root_derivatives)

# Get a list of all file paths with .bdf extension in the source directory
all_files_path = sorted(glob(f'{root_source}/*/*/*V2_eeg.vhdr*'), key=len)

# Define a dictionary with participant numbers as keys and a list of bad channels as values + a list of bad ica components

prtp_chs_out = {'154': ['T7'], '156': ['PO3', 'T7', 'PO4'], '157': ['PO3', 'T7', 'Pz'], '159': ['Pz', 'O1', 'PO3', 'T7', 'P7', 'CP1'], '166': ['PO3'], '169': [], '170': ['O1', 'PO3'], '171': ['CP6', 'PO3'], '172': ['Pz', 'PO3', 'CP5', 'T7', 'CP1'], '173': [], '175': ['Pz', 'P8', 'O1', 'O2', 'PO3', 'T7'], '176': ['PO3'], '177': ['PO3'], '178': ['P3', 'PO3', 'T7', 'Pz'], '179': ['P7', 'O2', 'PO3'], '180': ['CP6', 'PO3'], '181': ['P7', 'P3', 'PO3', 'CP2'], '183': ['PO3'], '184': ['T8', 'F7', 'F4', 'PO3', 'T7', 'F3'], '728': ['PO3'], '729': ['PO3'], '730': [], '731': ['Pz'], '732': ['Pz'], '733': ['PO3'], '734': ['PO3', 'Pz'], '735': [], '736': [], '737': [], '739': ['P7', 'T8', 'T7'], '740': ['P3'], '741': ['CP6', 'PO3', 'T8', 'PO4'], '742': ['P7', 'PO3', 'PO4'], '743': ['T8', 'PO3', 'Pz'], '744': ['Pz'], '746': ['PO4', 'Pz'], '747': ['O1'], '748': ['PO3'], '931': [], '932': ['P3', 'PO3'], '933': ['P3', 'PO3'], '934': ['Pz'], '935': ['PO3', 'Pz'], '937': [], '938': ['PO3'], '940': ['PO3', 'Pz'], '941': ['P7', 'O1'], '942': ['PO3'], '944': ['Pz'], '945': ['P3', 'PO3', 'T7'], '946': ['PO3', 'Pz'], '947': ['T8', 'Fp2'], '948': ['PO3', 'Pz'], '949': ['P3', 'F7', 'Pz'], '950': ['O2', 'P4', 'Oz', 'CP5'], '951': ['T8', 'Fp1', 'P4', 'PO3', 'T7', 'PO4'], '952': ['O2', 'PO3', 'Oz', 'Pz'], '953': ['T7'], '954': ['PO3'], '955': ['P4', 'PO3'], '956': ['P3', 'CP2', 'CP1'], '957': ['O1', 'P3', 'PO3'], '958': [], '155': ['Fp1', 'Fp2', 'PO3'], '160': ['PO3'], '161': [], '162': ['PO3', 'T7'], '163': [], '164': ['Fp1', 'Fp2', 'Pz', 'PO4'], '165': ['T7'], '167': ['Oz'], '174': ['PO3', 'PO4'], '182': ['P3'], '745': ['PO3'], '936': ['Fp1', 'Fp2', 'F7'], '939': []}

prtp_ica_out ={'154': [0, 2], '156': [0, 2], '157': [0, 1], '159': [0, 1], '162': [0, 2], '166': [1, 2], '169': [0, 1], '170': [3, 1], '171': [0, 1, 2], '172': [0, 3], '173': [1, 3], '175': [0, 1], '176': [0, 2], '177': [0, 1], '178': [0, 4], '179': [0, 2], '180': [1, 4], '181': [0, 2], '183': [0, 2, 3], '184': [1, 4], '728': [0, 2], '729': [0, 2], '730': [0, 2], '731': [2, 4], '732': [1, 0], '733': [0, 1], '734': [0, 5], '735': [14, 5, 2], '736': [1, 2], '737': [0, 1], '739': [1, 0], '740': [0, 2], '741': [0, 3, 1], '742': [0, 1], '743': [5, 17], '744': [0, 3], '746': [0, 5], '747': [8, 9], '748': [0, 3], '931': [0, 1], '932': [1, 4], '933': [0, 1], '934': [0, 2], '935': [1,3], '937': [0, 1], '938': [0, 1], '940': [0, 2], '941': [0, 1], '942': [0, 2], '944': [0, 1], '945': [0, 1], '946': [2, 3], '947': [2, 0, 1], '948': [0, 1], '949': [0, 1], '950': [0, 3, 4], '951': [2, 4], '952': [1,12], '953': [1, 2], '954': [0, 2, 1], '955': [0, 4, 3], '956': [1, 2], '957': [1, 5, 7], '958': [0, 1],'155': [4,6], '160': [1,3], '161': [2,4], '163': [2,3], '164': [6,10], '165': [3, 6], '167': [0, 3], '174': [4, 5], '182': [1, 26], '745': [2, 22], '936': [1,6], '939': [22, 17]}



In [6]:
# Extract the participant number from the file path
n_part = '155'
bidspath = BIDSPath(subject=n_part, task='socInfluenceV2', datatype='eeg', root=root_bids)
raw = read_raw_bids(bids_path=bidspath).load_data()

# Apply a bandpass filter to the raw data 
raw.filter(l_freq=1, h_freq=30, picks='all', n_jobs=-1)

Extracting parameters from C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\1-bids\sub-155\eeg\sub-155_task-socInfluenceV2_eeg.vhdr...
Setting channel info structure...
Reading events from C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\1-bids\sub-155\eeg\sub-155_task-socInfluenceV2_events.tsv.
Reading channel info from C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\1-bids\sub-155\eeg\sub-155_task-socInfluenceV2_channels.tsv.
Reading electrode coords from C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\1-bids\sub-155\eeg\sub-155_space-CapTrak_electrodes.tsv.
Reading 0 ... 779409  =      0.000 ...  1522.283 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidt

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  25 out of  33 | elapsed:    9.5s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:    9.6s finished


Measurement date,"March 30, 2023 15:26:00 GMT"
Experimenter,mne_anonymize
Participant,sub-155
Digitized points,35 points
Good channels,"31 EEG, 1 Stimulus"
Bad channels,T7
EOG channels,Not available
ECG channels,Not available
Sampling frequency,512.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [7]:
import matplotlib
matplotlib.use("Qt5Agg")
raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 2666x1401 with 4 Axes>

Channels marked as bad:
['T7']


In [20]:
raw.plot_psd()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Setting 393523 of 666112 (59.08%) samples to NaN, retaining 272589 (40.92%) samples.
Effective window size : 4.000 (s)
At least one good data span is shorter than n_per_seg, and will be analyzed with a shorter window than the rest of the file.
Plotting power spectral density (dB=True).


C:\Users\mfbpe\AppData\Local\Temp\ipykernel_44552\2847632005.py:1: FutureWarning: The value of `amplitude='auto'` will be removed in MNE 1.8.0, and the new default will be `amplitude=False`.
  raw.plot_psd()


<MNELineFigure size 1000x350 with 2 Axes>

In [8]:
write_raw_bids(raw, bidspath, format='BrainVision', allow_preload=True, overwrite=True)


Writing 'C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\bids\participants.tsv'...
Writing 'C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\bids\participants.json'...
Writing 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/bids/sub-151/eeg/sub-151_space-CapTrak_electrodes.tsv'...
Writing 'C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/bids/sub-151/eeg/sub-151_space-CapTrak_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: ['BAD_', 'selection/forced/authority/no', 'selection/forced/authority/yes', 'selection/forced/group/no', 'selection/forced/group/yes', 'selection/forced/indiv/no', 'selection/forced/indiv/yes', 'selection/free/authority/no', 'selection/free/authority/yes', 'selection/free/group/no', 'selection/free/group/yes', 'selection/free/indiv/n

C:\Users\mfbpe\AppData\Local\Temp\ipykernel_2448\3102110663.py:1: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(raw, bidspath, format='BrainVision', allow_preload=True, overwrite=True)


Writing 'C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\bids\sub-151\sub-151_scans.tsv'...
Wrote C:\Users\mfbpe\Desktop\DATA\2023_Social_Influence\bids\sub-151\sub-151_scans.tsv entry with eeg\sub-151_task-socInfluenceV2_eeg.vhdr.


BIDSPath(
root: C:/Users/mfbpe/Desktop/DATA/2023_Social_Influence/bids
datatype: eeg
basename: sub-151_task-socInfluenceV2_eeg.vhdr)

In [2]:

# Interpolate bad channels for participants in the prtp_chs_out list
if n_part in list(prtp_chs_out):
    raw.info['bads'] = prtp_chs_out[n_part]
    raw = raw.interpolate_bads()
else :
    noisy= NoisyChannels(raw)
    noisy.find_all_bads()
    bad_chs = noisy.get_bads()
    raw.info['bads'] = bad_chs
    raw = raw.interpolate_bads()

    prtp_chs_out[n_part]=bad_chs

# Perform Independent Component Analysis (ICA) on the copied data
ica = mne.preprocessing.ICA(random_state=21, max_iter='auto')
ica.fit(raw)

# Set the channel types for EOG channels based on the participant
raw.set_channel_types({'Fp1': 'eog', 'Fp2': 'eog'})

# Selecting bad components ICA for participants in the prtp_ica_out list or calculating bad components

if n_part in list(prtp_ica_out):
    eog_indices = prtp_ica_out[n_part]
else : 
    # Find ICs (Independent Components) that match the EOG pattern
    eog_indices, eog_scores = ica.find_bads_eog(raw, measure='correlation', threshold=0.5)
    prtp_ica_out[n_part]=eog_indices

ica.exclude = eog_indices

# Set the channel types back to EEG channels 
raw.set_channel_types({'Fp1': 'eeg', 'Fp2': 'eeg'})

# If there are EOG components to exclude, plot diagnostics and save the figures
if len(eog_indices) > 0:
    ica.apply(raw)
    fig = ica.plot_components(picks=np.arange(ica.n_components_), show = False) #create figures of the components with the one removed
    fig.savefig(f"{root_image}sub-{n_part}_SIV2_ICA.png")

del ica

# Save the preprocessed raw data
raw.save(f"sub-{n_part}_SIV2_raw.fif", overwrite=True)

del raw

# create the txt file

with open('list_bad_channels.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_chs_out) + '\n')
    
with open('list_bad_ica.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_ica_out) + '\n')    
